### Check Conversion

In this Notebook we will do a basic data quality check of the conversion, we will also see the immense difference in speed and data accessibility between geodatabases(standard geospatial datastructures)

The ability to subset specifically what we are looking for, and where we are looking for it without having to download a whole zip file and then open a larger vector dataset.

In [2]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import numpy as np
import random
import fiona
from shapely.geometry import box, Point
import xarray as xr
import dask.dataframe as dd
import pandas as pd
import shapely.wkb
import dask_geopandas as dgpd
import urllib
import s3fs

In [6]:
var = 'euseamap_2023_combined_current_and_wave_induced_energy_at_the_seabed'
val = ['Low energy', 'Moderate energy']

lat_range = [0, 80]
lon_range = [0, 80]

In [ ]:
vector_file ='../data/extracted_files/EUSeaMap_2023.gdb'

import geopandas as gpd

layers = fiona.listlayers(vector_file)
layer = layers[0]

gdf = gpd.read_file(vector_file, layer=layer)


gdf_filtered = gdf[gdf[var].isin(val)]
gdf_filtered = gdf_filtered.cx[lon_range[0]:lon_range[1], lat_range[0]:lat_range[1]]
# Plot the geodatabase data
fig, ax = plt.subplots(1, 1, figsize=(20, 10), subplot_kw={'projection': ccrs.PlateCarree()})
ax.coastlines()
gdf_filtered.plot(column=var, ax=ax, transform=ccrs.PlateCarree())
ax.set_title('Geodatabase')

plt.show()


In [ ]:


pqfile = "../data/temp_assets/converted_arco/EUSeaMap_2023_parquet.parquet"

ddf = dd.read_parquet(pqfile)

print(ddf)



In [ ]:
def read_geoparquet_s3(download_url):
        """
        Read a GeoParquet file from a specified S3 URL. Using dask geopandas

        :param download_url: The URL of the S3 asset to read.
        :type download_url: str
        :return: A Dask GeoDataFrame containing the GeoParquet data.
        :rtype: dask_geopandas.GeoDataFrame
        """
        parsed_url = urllib.parse.urlparse(download_url)
        net_loc = parsed_url.netloc
        object_key = parsed_url.path.lstrip('/')
        s3_uri = f's3://{object_key}'
        endpoint_url = f'https://{net_loc}'
        fs = s3fs.S3FileSystem(anon=True, use_ssl=True, client_kwargs={'endpoint_url': endpoint_url})
        ddf = dgpd.read_parquet(
            s3_uri,
            storage_options={'anon': True, 'client_kwargs' : {'endpoint_url': endpoint_url}}
        )
        return ddf

geopqurl ='https://s3.waw3-1.cloudferro.com/emodnet/emodnet_seabed_habitats/9985/EUSeaMap_2023_geoparquet.parquet'
pqurl2 = "https://s3.waw3-1.cloudferro.com/emodnet/emodnet_seabed_habitats/14098/eunis_surveymaps_geoparquet_edito.parquet/"

dgdf = read_geoparquet_s3(pqurl2)
print(dgdf) 
ddf1 = dgdf.partitions[0].compute()
print(ddf1.head())

gdf = gpd.GeoDataFrame(ddf1)
gdf

In [ ]:
import urllib
import dask_geopandas as dgpd
import dask.dataframe as dd
import s3fs
def read_parquet_from_s3(url):
    """
    Read a Parquet file from a specified S3 URL. The URL should be in the format 's3://bucket/key'.

    :param download_url: The URL of the S3 asset to read.
    :type download_url: str
    :return: A Dask dataframe containing the Parquet data.
    :rtype: dask.dataframe.DataFrame
    """
    parsed_url = urllib.parse.urlparse(url)
    net_loc = parsed_url.netloc
    object_key = parsed_url.path.lstrip('/')
    s3_uri = f's3://{object_key}'
    endpoint_url = f'https://{net_loc}'
    fs = s3fs.S3FileSystem(anon=True, use_ssl=True, client_kwargs={'endpoint_url': endpoint_url})
    ddf = dd.read_parquet(
        s3_uri,
        storage_options={'anon': True, 'client_kwargs' : {'endpoint_url': endpoint_url}},
        engine='pyarrow'
    )
    return ddf

download_url = 'https://s3.waw3-1.cloudferro.com/emodnet/emodnet_seabed_habitats/9985/EUSeaMap_2023_parquet.parquet'
pqurl2 = "https://s3.waw3-1.cloudferro.com/emodnet/emodnet_seabed_habitats/14098/eunis_surveymaps_geoparquet.parquet/"

ddf = read_parquet_from_s3(pqurl2)
ddf1 = ddf.partitions[0].compute()
print(ddf1.head())


In [ ]:
gdf = ddf.compute()

gdf_filtered = gdf.cx[lon_range[0]:lon_range[1], lat_range[0]:lat_range[1]]

print(len(gdf_filtered))

In [ ]:
#plot Allcomb


# gdf_filtered = gdf_filtered[gdf_filtered['Energy'] == 3 ]
# Plot the geodatabase data
fig, ax = plt.subplots(1, 1, figsize=(20, 10), subplot_kw={'projection': ccrs.PlateCarree()})
ax.coastlines()
gdf_filtered.plot(column=var, ax=ax, transform=ccrs.PlateCarree(), cmap='viridis', legend=True)
ax.set_title('parquet')
plt.show()

In [ ]:

zarr_file = '../data/temp_assets/converted_arco/EUSeaMap_2023.zarr'

zarr_url = 'https://s3.waw3-1.cloudferro.com/emodnet/emodnet_seabed_habitats/9985/EUSeaMap_2023.zarr'

ds = xr.open_dataset(zarr_url, engine='zarr')
ds

In [ ]:


dsplot = ds[var].sel(longitude=slice(lon_range[0], lon_range[1]), latitude=slice(lat_range[0], lat_range[1]))
#encoding = ds_gdb2zarr[var].attrs['categorical_encoding']

arrayval = ds[var].attrs['categorical_encoding'][val[0]]
dsplot = dsplot.where(dsplot == arrayval)

print(np.nanmax(dsplot.values))

In [ ]:
# Create the plot
fig = plt.figure(figsize=(12, 12))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()
plot = dsplot.plot(ax=ax, transform=ccrs.PlateCarree(), 
                            cmap='YlOrBr', add_colorbar=True)

# Add title and labels
ax.set_title(f'Locations {var} ', fontsize=16)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

# Open the Zarr dataset
ds_gdb2zarr = xr.open_dataset(zarr_file_folk1, engine='zarr')

# Specify the variable
var = 'Energy'

# Get the data array
data = ds_gdb2zarr[var].values

# Find where the array is greater than 0
mask = data > 0

# Plot the mask
plt.figure(figsize=(10, 6))
plt.imshow(mask, cmap='gray', interpolation='none')  # Show the mask as a binary plot
plt.colorbar(label="Greater than 0 (1=True, 0=False)")
plt.title(f"Locations where {var} > 0")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()